# Sequential Learning Project : sequential weighted scheme comparison

## Intoduction
In this report, we will try multiple weighting scheme for sequential learning problems. As a reminder, the sequential learning problem is the following :
- We have a set of $N$ "experts", or predictor, $N \in \mathbb{N}$
- We get the data "online", meaning that we do not have access directly to a batch : the examples (data points in $\mathbb{R}^d$) are coming iteratively in time.